In [1]:
# Load data as list of DataFrames
# put excel tables to "/data" folder in current directory

import os
import math
import pandas as pd

pd.set_option("display.max_rows", 100000)


df_list = list()
for f in sorted(os.listdir('./data')):
    if f[0] != '.':
        print(f)
        df_list.append(pd.read_excel(os.path.join('./data', f), index_col=0))

00.Fields & Master Data.xlsx
01.COLs.xlsx
02.Supply Orders.xlsx
03.Operations.xlsx
04.ResourceGroupPeriod.xlsx
05.Routrings.xlsx
06.RoutringSteps.xlsx


In [2]:
# Список колонок в каждой таблице

for df in df_list:
    print(list(df.columns))
    print()

['Таблица', 'Описание таблицы', 'Колонка', 'Описание колонки', 'Справочник', 'Unnamed: 6', 'Unnamed: 7']

['COLAlloc', 'Quantity', 'MinQuantity', 'MaxQuantity', 'HasSalesBudgetReservation', 'RequiresOrderCombination', 'NrOfActiveRoutingChainUpstream', 'SelectedShippingShop', 'Вид ГП', 'DeliveryType', 'ImgPlannedStatus', 'RoutingId', 'Name', 'ProductId', 'ProductName', 'LatestDesiredDeliveryDate', 'ProductSpecificationId', 'ResourceGroupIds']

['ID план. зак.', 'ProductId', 'Заказ/Позиция', 'Название продукта', 'Vid_Product', 'Кол-во', 'ID склада', 'Planned Status', 'Начало', 'Окончание', 'Крайняя дата', 'ProductIdFull', 'RoutingId', 'DownstreamCustomerOrders']

['Id', 'OperationDescription', 'SequenceNr', 'Разрешенные стд. рес. №', 'Start', 'End', 'ProductionTime', 'InputQuantity', 'Output quantity', 'SchedulingSpace', 'ResourceGroupId', 'OperationCode', 'RoutingStepId']

['ResourceGroupID', 'Id', 'AvailableCapacity', 'FreeCapacity', 'Start', 'HasFiniteCapacity']

['RoutingId', 'InputP

In [3]:
# Количество цехов

# В документе 0.Fields - 6 вариантов
print('Doc 1:', set(df_list[1]['SelectedShippingShop']))  # 01.COLs.xlsx - 5 вариантов
print('Doc 6:', set(df_list[-1]['IdPlant']))  # 06.RoutringSteps.xlsx - 8 вариантов (+ 0, 50)

Doc 1: {7, 11, 12, 13, 15}
Doc 6: {0, 6, 7, 11, 12, 13, 15, 50}


In [4]:
# Ресурсные группы по цехам
# Используем цеха № {7, 11, 12, 13, 15}

cur_df = df_list[-1]
for f in [7, 11, 12, 13, 15]:
    print('Цех',f, '\tресурсные группы:\n', set(cur_df[cur_df['IdPlant'] == f]['ResourceGroupId']))
    print()

Цех 7 	ресурсные группы:
 {'G_DSO2', 'G_ATO2', 'G_MFO2', 'G_UNRO2', 'G_CMO2', 'G_VAO2', 'G_OTGRO2', 'G_VSO2'}

Цех 11 	ресурсные группы:
 {'G_STG1T', 'G_AR10T', 'G_AR7T', 'G_AR3T', 'G_ARG1T', 'G_VTOT', 'G_AEIPT', 'G_UPAT', 'G_ANOG2T', 'G_TKT', 'G_OTGRT', 'G_ARG2T', 'G_AZPT'}

Цех 12 	ресурсные группы:
 {'G_OTGRG', 'G_ST200G', 'G_APORG', 'G_UPAG', 'G_APRRG', 'G_MPG'}

Цех 13 	ресурсные группы:
 {'G_OTGRH', 'G_APOR', 'G_ST203H', 'G_UPAH', 'G_STRS1H', 'G_DSH', 'G_ANGC1H', 'G_APPH', 'G_KPH', 'G_APRR', 'G_NTAH', 'G_ANGC3H', 'G_ANOH'}

Цех 15 	ресурсные группы:
 {'G_APPD', 'G_ANGCD', 'G_AOAD', 'G_APHKRD', 'G_APD', 'G_APGKRD', 'G_OTGRD', 'G_UPAD', 'G_NORD', 'G_ANOD', 'G_ST140D', 'G_NTAD', 'G_DSD', 'G_RSTD'}



In [5]:
# Calculate ResourceGroup load as percentage
# 2 days, 15:03:53.280 - это значит, что суммарная доступность агрегатов внутри группы 2*24 часа + 15 часов
# но свободно 1:55:07.587

# Рабочая таблица 04.ResourceGroupPeriod.xlsx
cur_df = df_list[4]

def fmt_datetime(dt):
    """ Formats datetime to int = minutes number"""
    #print(dt)
    try:
        arr = str(dt).split(', ')

        if 'day' in arr[0]:
            days = int(arr[0].split(' ')[0])
        else:
            days = 0

        if len(arr) > 1 or 'day' not in arr[0]:
            hrs, mins = arr[-1].split(':')[:2]
            hrs, mins = int(hrs), int(mins)
        else:
            hrs, mins = 0, 0

        #print('---', days, hrs, mins)
        #print(days*24*60 + hrs*60 + mins)
        res = days*24*60 + hrs*60 + mins
    except ValueError:
        #print(dt)
        res = None
    return res
    
    
# Форматируем данные в таблице, сохраняя их в отдельные столбцы
cur_df['av_temp'] = cur_df['AvailableCapacity'].apply(fmt_datetime)
cur_df['fr_temp'] = cur_df['FreeCapacity'].apply(fmt_datetime)

# load in percents
cur_df['load'] = round(((cur_df['av_temp'] - cur_df['fr_temp']) / cur_df['av_temp']) * 100)

# if available capacity == 0 mark cell as -2
# important for finite load SourceGroups - Source is not able to work
cur_df['load'][cur_df['load'].isna()] = -2

print(set(cur_df['load']))
print(cur_df['load'][cur_df['load'] == -1])
print()
print(cur_df['load'][cur_df['load'] == -2])


{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, -2.0}
Series([], Name: load, dtype: float64)

#
553   -2.0
554   -2.0
542   -2.0
549   -2.0
541   -2.0
552   -2.0
547   -2.0
570   -2.0
572   -2.0
551   -2.0
559   -2.0
546   -2.0
564   -2.0
571   -2.0
555   -2.0
560   -2.0
566   -2.0
562   -2.0
557   -2.0
569   -2.0
561   -2.0
543   -2.0
550   -2.0
544   -2.0
567   -2.0
548   -2.0
558   -2.0
576   -2.0
545   -2.0
540   -2.0
568   -2.0
575   -2.0
57

/home/andrey/2.TEXEL/scripts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# add colours marking
# давайте примем, что до 80% - красное, 80%-95% желтая, выше - зеленая
# да... то, что без ограничения мощности - HasFiniteCapacity=FALSE я бы отдельным цветом подсвечивал

from collections import Counter

def mark_color(w):
    if w == -2:
        return -2
    elif w == -1:
        return -1
    elif 0 <= w <= 80:
        return 0
    elif 80 < w < 95:
        return 1
    elif w >= 95:
        return 2
    else:
        print(type(w))

# add load_color column
cur_df['load_color'] = cur_df['load'].apply(mark_color)
cur_df['load_color'][cur_df['HasFiniteCapacity'] == 0] = -1
#Counter(cur_df['load'])
#Counter(cur_df['load_color'])

# save new columns to csv file
cur_df[['load', 'load_color']].to_csv('loading_data_table4_new.csv')

/home/andrey/2.TEXEL/scripts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
